# Quick Start - AGB Point Query

## Overview

This notebook walks us through the Above Ground Biomass (AGB) point query and gives a brief about how to send a point query to AGB Geospatial APIs.

The most basic query in AGB Geospatial APIs is the point query. Now you'll make a point query with the user interface we are going to get you started with the Geospatial APIs API by using it to do a point query:

The below query requests <i>Above Ground Biomass</i> values from Geospatial APIs layer 38000, Above Ground Biomass dataset, for a location somewhere in Castelo, Portugal – the coordinates 38.50/-9.10 (latitude/longitude).

Geospatial APIs returns about 3 rows of data, which are stored in the AGB_df dataframe.


### Setup
Before starting, you have to install IBM EI Geospatial python SDK using `pip3 install -U ibmpairs`.

<b>Note:</b> To run this notebook seamlessly, you have to first configure your Geospatial APIs' credentials in a file named `secrets.ini` in the below format:

```
[EI]
api.host = https://api.ibm.com/geospatial/run/na/core/v3
api.api_key = <Your Geospatial APIs API key>
api.tenant_id = <Your Geospatial APIs Tenant ID>
api.org_id = <Your Geospatial APIs Org ID>
```

Keep the secrets.ini file at an appropriate relative location of this notebook. For example, as specified in the below config.

```config.read('../../auth/secrets.ini')```

In [5]:
!pip install ibmpairs

In [6]:
import ibmpairs.client as client
import ibmpairs.query as query

# other imports
import pandas as pd
import configparser

Now, let's create a client object using the API_KEY, TENANT_ID (or CLIENT_ID) and ORG_ID to create an authenticated HTTP client Authentication token using 'ibmpairs.client' module, to use it in the subsequent steps.

In [7]:
config = configparser.RawConfigParser()
config.read('../../auth/secrets.ini')

EI_ORG_ID     = config.get('EI', 'api.org_id') 
EI_TENANT_ID  = config.get('EI', 'api.tenant_id') 
EI_API_KEY     = config.get('EI', 'api.api_key')

EI_client_v3 = client.get_client(org_id    = EI_ORG_ID,
                                  tenant_id = EI_TENANT_ID,
                                  api_key     = EI_API_KEY,
                                  version   = 3
                                )

2024-06-25 17:17:03 - paw - INFO - The client authentication method is assumed to be OAuth2.
2024-06-25 17:17:03 - paw - INFO - Legacy Environment is False
2024-06-25 17:17:03 - paw - INFO - The authentication api key type is assumed to be IBM EIS, because the api key prefix 'PHX' is present.
2024-06-25 17:17:05 - paw - INFO - Authentication success.
2024-06-25 17:17:05 - paw - INFO - HOST: https://api.ibm.com/geospatial/run/na/core/v3


The following code snippet submits the query for the AGB Data Layer <i>above-ground-biomass</i>  for years 2020, 2021 and 2022 using Geospatial APIs V3 query API.

Let's go ahead, define and submit a query.

In [8]:
query_json = { "name": "Above Ground Biomass - Portugal",
                          "layers": [ {  "id": "38000", "type": "raster"  } ], # Above Ground Biomass datalayer
                          "spatial":  {  "type": "point", "coordinates": [ 38.50342316, -9.10818912 ] },
                          "temporal": {  "intervals": [ { "start": "2020-01-01 00:00:00", "end": "2022-01-02 00:00:00"  } ] }
             }

results = query.submit(query_json, client=EI_client_v3, verify=False)
AGB_df = results.point_data_as_dataframe()
AGB_df['date'] = pd.to_datetime(AGB_df['timestamp'] * 1e6, errors = 'coerce')
AGB_df = AGB_df.drop(columns=['timestamp'])
AGB_df

2024-06-25 17:17:05 - paw - INFO - TASK: submit STARTING.
2024-06-25 17:17:08 - paw - INFO - TASK: submit COMPLETED.


,layer_id,layer_name,dataset,longitude,latitude,value,date
0,38000,above ground biomass,Above Ground Biomass,-9.108189,38.503423,85.0,2020-01-01
1,38000,above ground biomass,Above Ground Biomass,-9.108189,38.503423,90.0,2021-01-01
2,38000,above ground biomass,Above Ground Biomass,-9.108189,38.503423,22.0,2022-01-01


#### Understanding the sample

<b>Code:</b>

We start with various import statements as needed:

```
import ibmpairs.authentication as authentication
import ibmpairs.client as client
import ibmpairs.query as query

# other imports
import pandas as pd

import configparser
```

After the imports we create a client object and use an API_KEY, TENANT_ID (or CLIENT_ID) and an ORG_ID to create an authenticated HTTP client with version 3 of Geospatial APIs by specifying the version as '3'.

```
EI_client_v3 = client.get_client(org_id    = EI_ORG_ID,
                                  tenant_id = EI_TENANT_ID,
                                  api_key     = EI_API_KEY,
                                  version   = 3
                                )
```

This is a required step before you start doing queries but you only need to do it once. 

The most intersting part of the above example is the definition of the actual query JSON that we send to Geospatial APIs.

```
query_json = { "name": "Above Ground Biomass - Portugal",
                          "layers": [ {  "id": "38000", "type": "raster"  } ], # Above Ground Biomass datalayer
                          "spatial":  {  "type": "point", "coordinates": [ 38.50342316, -9.10818912 ] },
                          "temporal": {  "intervals": [ { "start": "2020-01-01 00:00:00", "end": "2022-01-02 00:00:00"  } ] }
             }
```

In general, the query_json object answers the following questions: <i>what?, where? and when?</i>. What we are requesting is specified by the value associated to `layers`. Here, we are requesting a single raster layer with Id 38000. Next we define the spatial coverage of the query with the spatial key. In the above code, we only request data for a single point in the format [latitude, longitude]. Note that longitudes in EI range from -180 to +180 degrees. Using values larger than +180 will lead to error messages. Similarly, latitudes range of course from -90 to +90 degrees. Finally we define a single time range via the temporal field.

Subsequently we submit the query to Geospatial APIs. As this is a point query, the result is returned directly from the submit method call:

```query_result = query.submit(query_json)```

Note that we don’t explicitly need to tell the query object to use the authenticated client we created previously as it finds it automatically.

Geospatial APIs returns the result of a point query as JSON data. We use a helper method to turn this data into a local data frame:

```AGB_df = results.point_data_as_dataframe()```

From this point on all the data is in a local data frame and we can operate on it as we would on any other data frame.

<b>Output:</b>

The output above displays us the data that's been queried for the layer with id <i>38000</i> which corresponds to <i>Above Ground Biomass</i> Data Layer for 3 years 2020, 2021 and 2022 respectively in a tabular form. Each row typically consists of layer_id, layer_name, dataset, latitude and longitude that's being requested, value of the Above Ground Biomass (AGB) for over that year with the date on which the value is being presented.

<b>Note:</b> Point queries such as the above are unique in that they instantly return a response. This makes them particularly suited to testing as well as exploration and experimentation. If unsure about the data you are interested in- its spatial coverage frequency, or temporal extent- start with a point query. Having said that, note that some advanced features – most notably <b>user defined functions <>`</b> are not available for point queries.